# Trigger Tower performance plots


   * [Electrons flat pT 2 - 100 GeV](#Electrons-flat-pt-2---100-GeV)
   * [Hadron Gun Flat Pt XX - YY GeV](#Hadron-Gun-Flat-Pt-XX---YY-GeV)



In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [3]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")



def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")


In [4]:
version = 'v33'

#      Particle('ele', PID.electron),
#                  Particle('ele_A', PID.electron, '1.4 < abseta < 1.7'),
#                  Particle('ele_B', PID.electron, '1.7 <= abseta <= 2.8'),
#                  Particle('ele_C', PID.electron, 'abseta > 2.8'),
#                  Particle('photon', PID.photon),
#                  Particle('photon_A', PID.photon, '1.4 < abseta < 1.7'),
#                  Particle('photon_B', PID.photon, '1.7 <= abseta <= 2.8'),
#                  Particle('photon_C', PID.photon, 'abseta > 2.8'),
                
particles = ''

sample_names = ['ele_flat2to100_PU0',
                'ele_flat2to100_PU200',
                'photonPt35_PU0',
                'photonPt35_PU200']




sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


samples_ele = [sample_ele_flat2to100_PU0,sample_ele_flat2to100_PU200]
samples_photon = [sample_gPt35_PU0,sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0,sample_hadronGun_PU200]



samples = []








# sample_gPt50_PU0 = Sample('photonPt50_PU0', 'Pt50 PU0', version)
# sample_gPt50_PU200 = Sample('photonPt50_PU200', 'Pt50 PU200', version)

samples_pt35 = [sample_gPt35_PU0, sample_gPt35_PU200]
labels_pt35 = ['pt35 PU0', 'pt35 PU200']
# sample_gPt50_PU0
# samples_pt50 = [sample_gPt50_PU0, sample_gPt50_PU200]
# labels_pt50 = ['pt50 PU0', 'pt50 PU200']



Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v33.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v33.root does not exist


In [5]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em']
particles = ['ele', 'eleA', 'eleB', 'eleC']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]

histo_df


,ele,eleA,eleB,eleC
DEF,"[<l1THistos.HistoEff instance at 0x118155128>,...","[<l1THistos.HistoEff instance at 0x1176e0d40>,...","[<l1THistos.HistoEff instance at 0x1176e0c68>,...","[<l1THistos.HistoEff instance at 0x1176e0128>,..."
DEF_em,"[<l1THistos.HistoEff instance at 0x1176e0e60>,...","[<l1THistos.HistoEff instance at 0x1176e0ef0>,...","[<l1THistos.HistoEff instance at 0x118155f38>,...","[<l1THistos.HistoEff instance at 0x11814a8c0>,..."


In [ ]:
def getHistograms(name, samples):
    return [histos.GenParticleHistos(name, sample.histo_file) for sample in samples]

In [ ]:

def getEfficiencies(particle, algo, sample):
    ret = histos.GenParticleHistos('h_eff_{}_{}_{}'.format(algo, particle, sample.label))
    dens = histos.GenParticleHistos('h_genPartsSel_{}'.format(particle), sample.histo_file)
    nums = histos.GenParticleHistos('h_genPartsEff_{}_{}'.format(algo, particle), sample.histo_file)
#     draw(nums.h_pt)
#     draw(dens.h_pt)
    ret.h_pt.Divide(nums.h_pt, dens.h_pt, 1, 1, 'B')
    ret.h_eta.Divide(nums.h_eta, dens.h_eta, 1, 1, 'B')
    return ret
    
    

In [ ]:
def normalize(histos, factors):
    global normalized_histos
    for idx in range(0, len(histos)):
        print histos[idx]
        if histos[idx] not in normalized_histos:
            histos[idx].Scale(1./factors[idx])
            normalized_histos.append(histos[idx])

## Electrons flat pt 2 - 100 GeV

In [6]:
samples = samples_ele
labels = [sample.label for sample in samples]
particle = 'ele'

# # Extract # of events for normalization
# nevents_ee = [histo.GetBinContent(3) for histo in hGen_reachedEE]
# print nevents_ee


### All Towers (no match)

In [7]:

hTT_all = [histos.TriggerTowerHistos('h_TT_all', sample.histo_file) for sample in samples]


In [8]:
hTT_pt = [histo.h_pt for histo in hTT_all]
drawSame(hTT_pt, labels, 'hist', logy=True)



In [9]:
hTT_eta = [histo.h_eta for histo in hTT_all]
drawSame(hTT_eta, labels, 'hist', logy=True)


In [10]:
hTT_HoE = [histo.h_HoE for histo in hTT_all]
drawSame(hTT_HoE, labels, 'hist', logy=True, norm=True)

### Towers matched to GEN particles

In [11]:

hTT_matched = [histos.TriggerTowerHistos('h_TT_{}'.format(particle), sample.histo_file) for sample in samples]


In [12]:
hTT_m_pt = [histo.h_pt for histo in hTT_matched]
#normalize(hTT_m_pt, nevents_ee)
drawSame(hTT_m_pt, labels, 'hist', logy=True)

In [13]:
hTT_m_eta = [histo.h_eta for histo in hTT_matched]
#normalize(hTT_m_eta, nevents_ee)
drawSame(hTT_m_eta, labels, 'hist', logy=True)

In [14]:
hTT_m_HoE = [histo.h_HoE for histo in hTT_matched]
#normalize(hTT_m_HoE, nevents_ee)
drawSame(hTT_m_HoE, labels, 'hist', logy=True)

### Trigger Tower resolution
#### Matching the highest pT TT in a DR=0.2

In [15]:
h_reso_TT = [histos.TriggerTowerResoHistos('h_reso_TT_{}'.format(particle), sample.histo_file) for sample in samples]


In [16]:
h_TT_ptRes = [histo.h_ptRes for histo in h_reso_TT]
# normalize(h_TT_ptRes, nevents_ee)
drawSame(h_TT_ptRes, labels, 'hist')

In [17]:
h_TT_ptResVpt = [histo.h_ptResVpt for histo in h_reso_TT]
drawSeveral(h_TT_ptResVpt, labels, 'COLZ',do_profile=True)


In [18]:
h_TT_ptResVpt_0 = [histo.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_0:
    histo.SetTitle('TT Pt reso (GeV) for 0 < pt < 10 GeV')
# normalize(h_TT_ptResVpt_0, nevents_ee)

drawSame(h_TT_ptResVpt_0, labels, 'hist')


In [19]:
h_TT_ptResVpt_1 = [histo.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_1:
    histo.SetTitle('TT Pt reso (GeV) for 10 < pt < 20 GeV')
# normalize(h_TT_ptResVpt_1, nevents_ee)

drawSame(h_TT_ptResVpt_1, labels, 'hist',norm=True)


In [20]:
h_TT_ptResVpt_2 = [histo.ProjectionY(uuid.uuid4().hex[:6], 11, 15) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_2:
    histo.SetTitle('TT Pt reso (GeV) for 20 < pt < 30 GeV')
# normalize(h_TT_ptResVpt_2, nevents_ee)

drawSame(h_TT_ptResVpt_2, labels, 'hist')


In [21]:
h_TT_ptResVpt_3 = [histo.ProjectionY(uuid.uuid4().hex[:6], 16, 20) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_3:
    histo.SetTitle('TT Pt reso (GeV) for 30 < pt < 40 GeV')
# normalize(h_TT_ptResVpt_3, nevents_ee)

drawSame(h_TT_ptResVpt_3, labels, 'hist',norm=True)


In [22]:
h_TT_etaRes = [histo.h_etaRes for histo in h_reso_TT]
# normalize(h_TT_etaRes, nevents_ee)
drawSame(h_TT_etaRes, labels, 'hist')
drawGaussFit(h_TT_etaRes[0], 3, -0.4, 0.4)

 FCN=8450.6 FROM MIGRAD    STATUS=CONVERGED      63 CALLS          64 TOTAL
                     EDM=6.03461e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.96432e+04   5.49433e+01   2.17987e+00  -6.08321e-07
   2  Mean        -5.11144e-05   6.67621e-05   2.99684e-06  -1.24836e-01
   3  Sigma        2.66515e-02   3.57824e-05   1.73770e-05   1.37520e-03


In [23]:
h_TT_phiRes = [histo.h_phiRes for histo in h_reso_TT]
# normalize(h_TT_phiRes, nevents_ee)
drawSame(h_TT_phiRes, labels, 'hist')
drawGaussFit(h_TT_phiRes[0], 3, -0.1, 0.1)
drawGaussFit(h_TT_phiRes[1], 3, -0.1, 0.1)


 FCN=3313.99 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=2.28148e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.66353e+04   4.97313e+01   1.15112e+00   1.04119e-05
   2  Mean         7.44496e-05   7.84869e-05   2.20657e-06   6.60432e+00
   3  Sigma        3.16982e-02   5.44115e-05   1.31555e-05   3.82204e-01
 FCN=790.263 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=8.43483e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.64287e+03   2.32814e+01   2.62884e-01  -5.73656e-05
   2  Mean         3.52389e-04   1.67996e-04   2.30696e-06  -1.76351e+00
   3  Sigma        3.17720e-02   

In [26]:
h_TT_ptResp = [histo.h_ptResp for histo in h_reso_TT]
# normalize(h_TT_ptRes, nevents_ee)
drawSame(h_TT_ptResp, labels, 'hist')

In [27]:
h_TT_ptRespVpt = [histo.h_ptRespVpt for histo in h_reso_TT]
drawSeveral(h_TT_ptRespVpt, labels, 'COLZ',do_profile=True)


In [ ]:
h_TT_ptResVpt_0 = [histo.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_0:
    histo.SetTitle('TT Pt reso (GeV) for 0 < pt < 10 GeV')
# normalize(h_TT_ptResVpt_0, nevents_ee)

drawSame(h_TT_ptResVpt_0, labels, 'hist')


In [28]:
h_TT_ptRespVeta = [histo.h_ptRespVeta for histo in h_reso_TT]
drawSeveral(h_TT_ptRespVeta, labels, 'COLZ',do_profile=True)


## Hadron Gun Flat Pt XX - YY GeV

In [24]:
samples = samples_hadrons
labels = [sample.label for sample in samples]
particle = 'pion'

# # Extract # of events for normalization
# nevents_ee = [histo.GetBinContent(3) for histo in hGen_reachedEE]
# print nevents_ee


### All Towers (no match)

In [25]:

hTT_all = [histos.TriggerTowerHistos('h_TT_all', sample.histo_file) for sample in samples]


In [ ]:
hTT_pt = [histo.h_pt for histo in hTT_all]
drawSame(hTT_pt, labels, 'hist', logy=True)



In [ ]:
hTT_eta = [histo.h_eta for histo in hTT_all]
drawSame(hTT_eta, labels, 'hist', logy=True)


In [ ]:
hTT_HoE = [histo.h_HoE for histo in hTT_all]
drawSame(hTT_HoE, labels, 'hist', logy=True, norm=True)

### Towers matched to GEN particles

In [ ]:

hTT_matched = [histos.TriggerTowerHistos('h_TT_{}'.format(particle), sample.histo_file) for sample in samples]


In [ ]:
hTT_m_pt = [histo.h_pt for histo in hTT_matched]
#normalize(hTT_m_pt, nevents_ee)
drawSame(hTT_m_pt, labels, 'hist', logy=True)

In [ ]:
hTT_m_eta = [histo.h_eta for histo in hTT_matched]
#normalize(hTT_m_eta, nevents_ee)
drawSame(hTT_m_eta, labels, 'hist', logy=True)

In [ ]:
hTT_m_HoE = [histo.h_HoE for histo in hTT_matched]
#normalize(hTT_m_HoE, nevents_ee)
drawSame(hTT_m_HoE, labels, 'hist', logy=True)

### Trigger Tower resolution
#### Matching the highest pT TT in a DR=0.2

In [ ]:
h_reso_TT = [histos.TriggerTowerResoHistos('h_reso_TT_{}'.format(particle), sample.histo_file) for sample in samples]


In [ ]:
h_TT_ptRes = [histo.h_ptRes for histo in h_reso_TT]
# normalize(h_TT_ptRes, nevents_ee)
drawSame(h_TT_ptRes, labels, 'hist', logy=True)

In [ ]:
h_TT_ptResVpt = [histo.h_ptResVpt for histo in h_reso_TT]
drawSeveral(h_TT_ptResVpt, labels, 'COLZ',do_profile=True)


In [ ]:
h_TT_ptResVpt_0 = [histo.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_0:
    histo.SetTitle('TT Pt reso (GeV) for 0 < pt < 10 GeV')
# normalize(h_TT_ptResVpt_0, nevents_ee)

drawSame(h_TT_ptResVpt_0, labels, 'hist', norm=True)


In [ ]:
h_TT_ptResVpt_1 = [histo.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_1:
    histo.SetTitle('TT Pt reso (GeV) for 10 < pt < 20 GeV')
# normalize(h_TT_ptResVpt_1, nevents_ee)

drawSame(h_TT_ptResVpt_1, labels, 'hist',norm=True)


In [ ]:
h_TT_ptResVpt_2 = [histo.ProjectionY(uuid.uuid4().hex[:6], 11, 15) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_2:
    histo.SetTitle('TT Pt reso (GeV) for 20 < pt < 30 GeV')
# normalize(h_TT_ptResVpt_2, nevents_ee)

drawSame(h_TT_ptResVpt_2, labels, 'hist')


In [ ]:
h_TT_ptResVpt_3 = [histo.ProjectionY(uuid.uuid4().hex[:6], 16, 20) for histo in h_TT_ptResVpt]
for histo in h_TT_ptResVpt_3:
    histo.SetTitle('TT Pt reso (GeV) for 30 < pt < 40 GeV')
# normalize(h_TT_ptResVpt_3, nevents_ee)

drawSame(h_TT_ptResVpt_3, labels, 'hist',norm=True)


In [ ]:
h_TT_etaRes = [histo.h_etaRes for histo in h_reso_TT]
# normalize(h_TT_etaRes, nevents_ee)
drawSame(h_TT_etaRes, labels, 'hist')
drawGaussFit(h_TT_etaRes[0], 3, -0.4, 0.4)

In [ ]:
h_TT_phiRes = [histo.h_phiRes for histo in h_reso_TT]
# normalize(h_TT_phiRes, nevents_ee)
drawSame(h_TT_phiRes, labels, 'hist')
drawGaussFit(h_TT_phiRes[0], 3, -0.1, 0.1)
drawGaussFit(h_TT_phiRes[1], 3, -0.1, 0.1)


In [ ]:
drawSame([hCl3Dbest_ptResVpt_0[0], h_TT_ptResVpt_0[0]], ['CL3D (highest pt)', 'TT'], options="hist", norm=True)

In [ ]:
drawSame([hCl3Dbest_ptResVpt_1[0], h_TT_ptResVpt_1[0]], ['CL3D (highest pt)', 'TT'], norm=True, max_y=0.08)

In [ ]:
drawSame([hCl3Dbest_ptResVpt_1[1], h_TT_ptResVpt_1[1]], ['CL3D (highest pt)', 'TT'], norm=True, max_y=0.08)

In [ ]:
drawSame([hCl3D_ptResVpt_2[0], h_TT_ptResVpt_2[0]], ['CL3D in cone', 'TT'], norm=True)

In [ ]:
drawSame([hCl3Dbest_ptResVpt_3[0], h_TT_ptResVpt_3[0]], ['CL3D (highest pt)', 'TT'], norm=True, max_y=0.035)

In [ ]:
drawSame([hCl3Dbest_ptResVpt_3[1], h_TT_ptResVpt_3[1]], ['CL3D (highest pt)', 'TT'], norm=True, max_y=0.035)

#### Building a 3x3 Cluster around the highest-Pt trigger Tower

In [ ]:
h_reso_TTCL = [histos.TriggerTowerResoHistos('h_reso_TTCl_{}'.format(particle), sample.histo_file) for sample in samples]


In [ ]:
h_TTCL_ptRes = [histo.h_ptRes for histo in h_reso_TTCL]
normalize(h_TTCL_ptRes, nevents_ee)
drawSame(h_TTCL_ptRes, labels, 'hist', logy=True)

In [ ]:
h_TTCl_ptResVpt = [histo.h_ptResVpt for histo in h_reso_TTCL]
drawSeveral(h_TTCl_ptResVpt, labels, 'COLZ',do_profile=True)


In [ ]:
h_TTCl_ptResVpt_0 = [histo.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in h_TTCl_ptResVpt]
for histo in h_TTCl_ptResVpt_0:
    histo.SetTitle('TT Pt reso (GeV) for 0 < pt < 10 GeV')
normalize(h_TTCl_ptResVpt_0, nevents_ee)

drawSame(h_TTCl_ptResVpt_0, labels, 'hist')


In [ ]:
drawSame([hCl3Dbest_ptResVpt_0[0], h_TTCl_ptResVpt_0[0], hCl3D_ptResVpt_0[0], h_TT_ptResVpt_0[0]], 
         ['CL3D  highest pT', 'TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
drawSame([h_TTCl_ptResVpt_0[1], hCl3D_ptResVpt_0[1], h_TT_ptResVpt_0[1]], ['TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
h_TTCl_ptResVpt_1 = [histo.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in h_TTCl_ptResVpt]
for histo in h_TTCl_ptResVpt_1:
    histo.SetTitle('TT Pt reso (GeV) for 10 < pt < 20 GeV')
normalize(h_TTCl_ptResVpt_1, nevents_ee)

drawSame(h_TTCl_ptResVpt_1, labels, 'hist')


In [ ]:
drawSame([hCl3Dbest_ptResVpt_1[0], h_TTCl_ptResVpt_1[0], hCl3D_ptResVpt_1[0], h_TT_ptResVpt_1[0]], 
         ['CL3D  highest pT', 'TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
drawSame([hCl3Dbest_ptResVpt_1[1], h_TTCl_ptResVpt_1[1], hCl3D_ptResVpt_1[1], h_TT_ptResVpt_1[1]], 
         ['CL3D  highest pT', 'TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
h_TTCl_ptResVpt_2 = [histo.ProjectionY(uuid.uuid4().hex[:6], 11, 15) for histo in h_TTCl_ptResVpt]
for histo in h_TTCl_ptResVpt_2:
    histo.SetTitle('TT Pt reso (GeV) for 20 < pt < 30 GeV')
normalize(h_TTCl_ptResVpt_2, nevents_ee)

drawSame(h_TTCl_ptResVpt_2, labels, 'hist')


In [ ]:
drawSame([hCl3Dbest_ptResVpt_2[0], h_TTCl_ptResVpt_2[0], hCl3D_ptResVpt_2[0], h_TT_ptResVpt_2[0]], 
         ['CL3D  highest pT', 'TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
drawSame([hCl3Dbest_ptResVpt_2[1], h_TTCl_ptResVpt_2[1], hCl3D_ptResVpt_2[1], h_TT_ptResVpt_2[1]], 
         ['CL3D  highest pT', 'TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
h_TTCl_ptResVpt_3 = [histo.ProjectionY(uuid.uuid4().hex[:6], 16, 20) for histo in h_TTCl_ptResVpt]
for histo in h_TTCl_ptResVpt_3:
    histo.SetTitle('TT Pt reso (GeV) for 30 < pt < 40 GeV')
normalize(h_TTCl_ptResVpt_3, nevents_ee)

drawSame(h_TTCl_ptResVpt_3, labels, 'hist')


In [ ]:
drawSame([hCl3Dbest_ptResVpt_3[0], h_TTCl_ptResVpt_3[0], hCl3D_ptResVpt_3[0], h_TT_ptResVpt_3[0]], 
         ['CL3D  highest pT', 'TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
drawSame([hCl3Dbest_ptResVpt_3[1], h_TTCl_ptResVpt_3[1], hCl3D_ptResVpt_3[1], h_TT_ptResVpt_3[1]], 
         ['CL3D  highest pT', 'TT CL', 'CL3D in cone', 'TT'], norm=True)

In [ ]:
h_TTCl_etaRes = [histo.h_etaRes for histo in h_reso_TTCL]
normalize(h_TTCl_etaRes, nevents_ee)
drawSame(h_TTCl_etaRes, labels, 'hist')

In [ ]:
h_TTCl_etalwRes = [histo.h_etalwRes for histo in h_reso_TTCL]
normalize(h_TTCl_etalwRes, nevents_ee)
drawSame(h_TTCl_etalwRes, labels, 'hist')

In [ ]:
drawSame([h_TTCl_etaRes[0],h_TT_etaRes[0]], 
         ['TT cl', 'TT'])

In [ ]:
h_TTCl_phiRes = [histo.h_phiRes for histo in h_reso_TTCL]
normalize(h_TTCl_phiRes, nevents_ee)
drawSame(h_TTCl_phiRes, labels, 'hist')

In [ ]:
drawSame([h_TTCl_phiRes[0],h_TT_phiRes[0]], 
         ['TT cl', 'TT'])